In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão de Python Neste Jupyter Notebook:', python_version())

# usaremos o filtro 'warning' para deixar mais limpo.
import warnings
warnings.filterwarnings('ignore')

### Heat Maps

In [ ]:
import json, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def read_json(f):
    with open(f) as f:

        return json.load(f)

def verify_keys(d, **kwargs):
    data = d[0].items()
    k1 = set([tup[0] for tup in data])

    s = kwargs.items()
    k2 = set([tup[1] for tup in s])

    return list(k1.intersection(k2))

    
def build_ls(k, d):

    return [{k: row[k] for k in (keys)} for row in d]

def get_rows(d, n):
    [print(row) for i, row in enumerate(d) if i < n]

def conv_float(d):

    return [dict([k, float(v)] for k, v in row.items()) for row in d]

if __name__ == "__main__":
    f= 'data/wrangled.json'
    data = read_json(f)
    keys = verify_keys(data, 
                       c1 = 'sale',
                       c2 = 'quan',
                       c3 = 'disc',
                       c4 = 'prof')

    heat = build_ls(keys, data)
    print ('1st row in "heat":')
    get_rows(heat, 1)

    heat = conv_float(heat)
    print ('\n1st row in "heat" converted to float:')
    get_rows(heat, 1)

    df = pd.DataFrame(heat)

    plt.figure()
    sns.heatmap(df.corr(), 
                annot = True,
                cmap = 'OrRd')
    plt.show()

### Principal Component Analysis

In [ ]:
import matplotlib.pyplot as plt, pandas as pd
import numpy as np, json, random as rnd
from sklearn.preprocessing import StandardScaler
from pandas.plotting import parallel_coordinates

def read_json(f):
    with open(f) as f:

        return json.load(f)

def unique_features(k, d):

    return list(set([dic[k] for dic in d]))

def sire_features(k, d):
    return [{k: row[k] for k in (k)} for row in d]

def sire_numeric(k, d):
    s = conv_float(sire_features(k, d))

    return s

def sire_sample(k, v, d, m):
    indices = np.arange(0, len(d), 1)
    s = [d[i] for i in indices if d[i][k] == v]
    n = len(s)
    num_keys = ['sale', 'quan', 'disc', 'prof']
    
    for i, row in enumerate(s):
        for k in num_keys:
            row[k] = float(row[k])
    
    s = rnd_sample(m, len(s), s)
    
    return (s, n)

def rnd_sample(m, n, d):
    indices = sorted(rnd.sample(range(n), m))
    
    return [d[i] for i in indices]

def conv_float(d):
    
    return [dict([k, float(v)] for k, v in row.items()) for row in d]

if __name__ == "__main__":
    f = 'data/wrangled.json'
    data = read_json(f)
    segm = unique_features('segm', data)
    print ('classes in "segm" feature:')
    print (segm)
    
    keys = ['sale', 'quan', 'disc', 'prof', 'segm']
    features = sire_features(keys, data)
    num_keys = ['sale', 'quan', 'disc', 'prof']
    numeric_data = sire_numeric(num_keys, features)
    
    k, v = "segm", "Home Office"
    m = 100
    s_home = sire_sample(k, v, features, m)
    
    v = "Consumer"
    s_cons = sire_sample(k, v, features, m)
    
    v = "Corporate"
    s_corp = sire_sample(k, v, features, m)
    print ('\nHome Office slice:', s_home[1])
    print('Consumer slice:', s_cons[1])
    print ('Coporate slice:', s_corp[1])
    print ('sample size:', m)
    
    df_home = pd.DataFrame(s_home[0])
    df_cons = pd.DataFrame(s_cons[0])
    df_corp = pd.DataFrame(s_corp[0])
    frames = [df_home, df_cons, df_corp]
    result = pd.concat(frames)
    
    plt.figure()
    
    parallel_coordinates(result, 
                         'segm',
                         color = ['orange','lime','fuchsia'])
    df = pd.DataFrame(numeric_data)
    X = df.ix[:].values
    X_std = StandardScaler().fit_transform(X)
    mean_vec = np.mean(X_std, axis=0)
    cov_mat = np.cov(X_std.T)
    print ('\ncovariance matrix:\n', cov_mat)

    eig_vals, eig_vecs = np.linalg.eig(cov_mat)
    print ('\nEigenvectors:\n', eig_vecs)
    print ('\nEigenvalues:\n', np.sort(eig_vals)[::-1])
    
    tot = sum(eig_vals)
    var_exp = [(i / tot)*100 for i in sorted(eig_vals,
                                             reverse = True)]
    print ('\nvariance explained:\n', var_exp)
    
    corr_mat = np.corrcoef(X.T)
    print ('\ncorrelation matrix:\n', corr_mat)
    
    eig_vals, eig_vecs = np.linalg.eig(corr_mat)
    print ('\nEigenvectors:\n', eig_vecs)
    print ('\nEigenvalues:\n', np.sort(eig_vals)[::-1])
    
    tot = sum(eig_vals)
    var_exp = [(i / tot)*100 for i in sorted(eig_vals,
                                             reverse = True)]
    print ('\nvariance explained:\n', var_exp)
    
    cum_var_exp = np.cumsum(var_exp)
    fig, ax = plt.subplots()
    labels = ['PC1', 'PC2', 'PC3', 'PC4']
    width = 0.35
    index = np.arange(len(var_exp))
    ax.bar(index, var_exp,
           color = ['fuchsia', 'lime', 'thistle', 'thistle'])

    for i, v in enumerate(var_exp):
        v = round(v, 2)
        val = str(v) + '%'
        ax.text(i, v+0.5, val, 
                ha = 'center', 
                color = 'b',
                fontsize = 9,
                fontweight = 'bold')
    
    plt.xticks(index, labels)
    plt.title('Variance Explained')
    plt.show()

### Speed Simulation

In [ ]:
import json, humanfriendly as hf
from time import clock

def read_json(f):
    with open(f) as f:
        return json.load(f)

def mk_gen(k, d):
    for row in d:
        dic = {}
        for key in k:
            dic[key] = float(row[key])
        yield dic

def conv_float(keys, d):
    return [dict([k, float(v)] for k, v in row.items()
                 if k in keys) for row in d]

if __name__ == "__main__":
    f = 'data/wrangled.json'
    data = read_json(f)
    keys = ['sale', 'quan', 'disc', 'prof']
    print ('create, convert, and display list:')
    start = clock()
    data = conv_float(keys, data)
    for i, row in enumerate(data):
        if i < 5:
            print (row)
    end = clock()
    elapsed_ls = end - start
    print (hf.format_timespan(elapsed_ls, detailed=True))
    print ('\ncreate, convert, and display generator:')
    start = clock()
    generator = mk_gen(keys, data)
    for i, row in enumerate(generator):
        if i < 5:
            print (row)
    end = clock()
    elapsed_gen = end - start
    print (hf.format_timespan(elapsed_gen, detailed=True))
    speed = round(elapsed_ls / elapsed_gen, 2)
    print ('\ngenerator is', speed, 'times faster')

### Big Data

In [ ]:
import json, csv

def read_dat(h, f):
    return csv.DictReader((line.replace('::', ':')
                           for line in open(f)),
                          delimiter=':', fieldnames=h,
                          quoting=csv.QUOTE_NONE)

def gen_dict(d):
    for row in d:
        yield dict(row)

def dump_json(f, l, d):
    f = open(f, 'w')
    f.write('[')
    for i, row in enumerate(d):
        j = json.dumps(row)
        f.write(j)
        if i < l - 1:
            f.write(',')
        else:
            f.write(']')
    f.close()

def read_json(f):
    with open(f) as f:
        return json.load(f)

def display(n, f):
    for i, row in enumerate(f):
        if i < n:
            print (row)
    print()

if __name__ == "__main__":
    print ('... sizing data ...\n')
    u_dat = 'data/ml-1m/users.dat'
    m_dat = 'data/ml-1m/movies.dat'
    r_dat = 'data/ml-1m/ratings.dat'
    unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
    mnames = ['movie_id', 'title', 'genres']
    rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
    users = read_dat(unames, u_dat)
    ul = len(list(gen_dict(users)))
    movies = read_dat(mnames, m_dat)
    ml = len(list(gen_dict(movies)))
    ratings = read_dat(rnames, r_dat)
    rl = len(list(gen_dict(ratings)))
    print ('size of datasets:')
    print ('users', ul)
    print ('movies', ml)
    print ('ratings', rl)
    print ('\n... dumping data ...\n')
    users = read_dat(unames, u_dat)
    users = gen_dict(users)
    movies = read_dat(mnames, m_dat)
    movies = gen_dict(movies)
    ratings = read_dat(rnames, r_dat)
    ratings = gen_dict(ratings)
    uf = 'data/users.json'
    dump_json(uf, ul, users)
    mf = 'data/movies.json'
    dump_json(mf, ml, movies)
    rf = 'data/ratings.json'
    dump_json(rf, rl, ratings)
    print ('\n... verifying data ...\n')
    u = read_json(uf)
    m = read_json(mf)
    r = read_json(rf)
    n = 1
    display(n, u)
    display(n, m)
    display(n, r)

In [ ]:
import json, numpy as np

def read_json(f):
    with open(f) as f:
        return json.load(f)

def dump_json(f, d):
    with open(f, 'w') as fout:
        json.dump(d, fout)    

def display(n, d):
    [print (row) for i,row in enumerate(d) if i < n]

def get_indx(k, d):
    return [row[k] for row in d if 'null' in row]

def get_data(k, l, d):
    return [row for i, row in enumerate(d) if row[k] in l]

def get_unique(key, d):
    s = set()
    for row in d:
        for k, v in row.items():
            if k in key:
                s.add(v)
    return np.sort(list(s))

if __name__ == "__main__":
    mf = 'data/movies.json'
    m = read_json(mf)
    n = 20
    display(n, m)
    print ()
    indx = get_indx('movie_id', m)
    for row in m:
        if row['movie_id'] in indx:
            row['title'] = row['title'] + ':' + row['genres']
            row['genres'] = row['null'][0]
            del row['null']
        title = row['title'].split(" ")
        year = title.pop()
        year = ''.join(c for c in year if c not in '()')
        row['title'] = ' '.join(title)
        row['year'] = year
    data = get_data('movie_id', indx, m)
    n = 2
    display(n, data)
    s = get_unique('year', m)
    print ('\n', s, '\n')
    rec = get_data('year', ['Assignment'], m)
    print (rec[0])
    rec = get_data('year', ["L'Associe1982"], m)
    print (rec[0], '\n')
    b1, b2, cnt = False, False, 0
    for row in m:
        if row['movie_id'] in ['1001']:
            row['year'] = '1982'
            print (row)
            b1 = True
        elif row['movie_id'] in ['2382']:
            row['title'] = 'Police Academy 5: Assignment: Miami Beach'
            row['genres'] = 'Comedy'
            row['year'] = '1988'
            print (row)
            b2 = True
        elif b1 and b2: break
        cnt += 1
    print ('\n', cnt, len(m))
    mf = 'data/cmovies.json'    
    dump_json(mf, m)
    m = read_json(mf)
    display(n, m)

In [ ]:
import json, numpy as np, sys, os, humanfriendly as hf
from time import clock
sys.path.append(os.getcwd()+'/classes')
import conn

def read_json(f):
    with open(f) as f:
        return json.load(f)

def get_column(A, v):
    return [A_i[v] for A_i in A]

def remove_nr(v1, v2):
    set_v1 = set(v1)
    set_v2 = set(v2)
    diff = list(set_v1 - set_v2)
    return diff

def get_info(*args):
    a = [arg for arg in args]
    ratings = [int(row[a[0][1]]) for row in a[2] if row[a[0][0]] == a[1]]
    uids = [row[a[0][3]] for row in a[2] if row[a[0][0]] == a[1]]
    title = [row[a[0][2]] for row in a[3] if row[a[0][0]] == a[1]]
    age = [int(row[a[0][4]]) for col in uids for row in a[4] if col == row[a[0][3]]]
    gender = [row[a[0][5]] for col in uids for row in users if col == row[a[0][3]]]
    return (ratings, title[0], uids, age, gender)

def generate(k, v, r, m, u):
   for i, mid in enumerate(v):
       dic = {}
       rec = get_info(k, mid, r, m, u)
       dic = {'_id':i, 'mid':mid, 'title':rec[1], 'avg_rating':np.mean(rec[0]),
              'n_ratings':len(rec[0]), 'avg_age':np.mean(rec[3]),
              'M':rec[4].count('M'), 'F':rec[4].count('F')}
       dic['avg_rating'] = round(float(str(dic['avg_rating'])[:6]),2)
       dic['avg_age'] = round(float(str(dic['avg_age'])[:6]))
       yield dic

def gen_ls(g):
    for i, row in enumerate(g):
        yield row

if __name__ == "__main__":
    print ('... creating datasets ...\n')
    m = 'data/cmovies.json'
    movies = np.array(read_json(m))
    r = 'data/ratings.json'
    ratings = np.array(read_json(r))
    r = 'data/users.json'
    users = np.array(read_json(r))
    print ('... creating movie indicies vector data ...\n')
    mv = get_column(movies, 'movie_id')
    rv = get_column(ratings, 'movie_id')
    print ('... creating unrated movie indicies vector ...\n')
    nrv = remove_nr(mv, rv)
    diff = [int(row) for row in nrv]
    print (np.sort(diff), '\n')
    new_mv = [x for x in mv if x not in nrv]
    mid = '1'
    keys = ('movie_id', 'rating', 'title', 'user_id', 'age', 'gender')
    stats = get_info(keys, mid, ratings, movies, users)
    avg_rating = np.mean(stats[0])
    avg_age = np.mean(stats[3])
    n_ratings = len(stats[0])
    title = stats[1]
    M, F = stats[4].count('M'), stats[4].count('F')
    print ('avg rating for:', end=' "')
    print (title + '" is', round(avg_rating, 2), end=' (')
    print (n_ratings, 'ratings)\n')
    gen = generate(keys, new_mv, ratings, movies, users)
    gls = gen_ls(gen)
    obj = conn.conn('test')
    db = obj.getDB()
    movie_info = db.movie_info
    movie_info.drop()
    print ('... saving movie_info to MongoDB ...\n')
    start = clock()
    for row in gls:
        movie_info.insert(row)
    end = clock()
    elapsed_ls = end - start
    print (hf.format_timespan(elapsed_ls, detailed=True))

In [ ]:
import sys, os, json, humanfriendly as hf
from time import clock
sys.path.append(os.getcwd() + '/classes')
import conn

def read_json(f):
    with open(f) as f:
        return json.load(f)

def create_db(c, d):
    c = db[c]
    c.drop()
    for i, row in enumerate(d):
        row['_id'] = i
        c.insert(row)

if __name__ == "__main__":
    u = read_json('data/users.json')
    m = read_json('data/cmovies.json')
    r = read_json('data/ratings.json')
    obj = conn.conn('test')
    db = obj.getDB()
    print ('... creating MongoDB collections ...\n')
    start = clock()
    create_db('users', u)
    create_db('movies', m)
    create_db('ratings', r)
    end = clock()
    elapsed_ls = end - start
    print (hf.format_timespan(elapsed_ls, detailed=True))

In [ ]:
import sys, os
sys.path.append(os.getcwd() + '/classes')
import conn

def match_item(k, v, d):
    pipeline = [ {'$match' : { k : v }} ]
    q = db.command('aggregate',d,pipeline=pipeline)
    return q

if __name__ == "__main__":
    obj = conn.conn('test')
    db = obj.getDB()
    movie = 'Toy Story'
    q = match_item('title', movie, 'movie_info')
    r = q['result'][0]
    print (movie, 'document:')
    print (r)
    print ('average rating', r['avg_rating'], '\n')
    user_id = '3'
    print ('*** user', user_id, '***')
    q = match_item('user_id', user_id, 'users')
    r = q['result'][0]    
    print ('age', r['age'], 'gender', r['gender'], 'occupation',\
          r['occupation'], 'zip', r['zip'], '\n')
    print ('*** "user 3" movie ratings of 5 ***')
    q = match_item('user_id', user_id, 'ratings')
    mid = q['result']
    for row in mid:
        if row['rating'] == '5':
            q = match_item('movie_id', row['movie_id'], 'movies')
            title = q['result'][0]['title']
            genre = q['result'][0]['genres']
            print (row['movie_id'], title, genre)
    mid = '1136'
    q = match_item('mid', mid, 'movie_info')
    title = q['result'][0]['title']
    avg_rating = q['result'][0]['avg_rating']
    print ()
    print ('"' + title + '"', 'average rating:', avg_rating)

In [ ]:
import sys, os
sys.path.append(os.getcwd() + '/classes')
import conn

def stages(k, v, r, d):
    pipeline = [ {'$match' : { '$and' : [ { k : v },
                   {'rating':{'$eq':r} }] } },
                 {'$project' : {
                     '_id' : 1,
                     'user_id' : 1,
                     'movie_id' : 1,
                     'rating' : 1 } },
                 {'$limit' : 100}]
    q = db.command('aggregate',d,pipeline=pipeline)
    return q

def match_item(k, v, d):
    pipeline = [ {'$match' : { k : v }} ]
    q = db.command('aggregate',d,pipeline=pipeline)
    return q

if __name__ == "__main__":
    obj = conn.conn('test')
    db = obj.getDB()
    u = '3'
    r = '5'
    q = stages('user_id', u, r, 'ratings')
    result = q['result']
    print ('ratings of', r, 'for user ' + str(u) + ':')
    for i, row in enumerate(result):
        print (row)
    n = i+1
    print ()
    print (n, 'associated movie titles:')
    for i, row in enumerate(result):
        q = match_item('movie_id', row['movie_id'], 'movies')
        r = q['result'][0]
        print (r['title'])

### Twitter

In [ ]:
import json

if __name__ == '__main__':
    consumer_key = 'qIhSaP3SDMjYTgQSHMnflOte0'
    consumer_secret = '7M4HGS8iPmma0tsC6KGMhqtlR4tDBl1YWPu1UtIMT5mvRNQw35'
    access_token = '3417237687-G6xlBPoHYQclCUhLoULgL6ubwiDjLmFUp1dEEqi'
    access_encrypted = 'zCDKTOHNFa31nBEGOhbttbA5RWX6lw7NR5jDsVJa3d6bh'
    data = {}
    data['ck'] = consumer_key
    data['cs'] = consumer_secret
    data['at'] = access_token
    data['ae'] = access_encrypted
    json_data = json.dumps(data)
    header = '[\n'
    ender = ']'
    obj = open('data/credentials.json', 'w')
    obj.write(header)
    obj.write(json_data + '\n')
    obj.write(ender)
    obj.close()

In [ ]:
from TwitterSearch import *
import json, sys

class twitSearch:
    def __init__(self, cred, ls, limit):
        self.cred = cred
        self.ls = ls
        self.limit = limit
    def search(self):
        num = 0
        dt = []
        dic = {}
        try:
            tso = TwitterSearchOrder()
            tso.set_keywords(self.ls)
            tso.set_language('en')
            tso.set_include_entities(False)
            ts = TwitterSearch(
                consumer_key = self.cred[0]['ck'],
                consumer_secret = self.cred[0]['cs'],
                access_token = self.cred[0]['at'],
                access_token_secret = self.cred[0]['ae']
                )
            for tweet in ts.search_tweets_iterable(tso):
                if num <= self.limit:
                    dic['_id'] = num
                    dic['tweeter'] = tweet['user']['screen_name']
                    dic['tweet_text'] = tweet['text']
                    dt.append(dic)
                    dic = {}
                else:
                    break
                num += 1
        except TwitterSearchException as e:
            print (e)
        return dt

def get_creds():
    with open('data/credentials.json') as json_data:
        d = json.load(json_data)
        json_data.close()
    return d

def write_json(f, d):
    with open(f, 'w') as fout:
        json.dump(d, fout)

def translate():
    return dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

def read_json(f):
    with open(f) as f:
        return json.load(f)

if __name__ == '__main__':
    cred = get_creds()
    ls = ['machine', 'learning']
    limit = 10
    obj = twitSearch(cred, ls, limit)
    data = obj.search()
    f = 'data/TwitterSearch.json'
    write_json(f, data)
    non_bmp_map = translate()
    print ('twitter data:')
    for row in data:
        row['tweet_text'] = str(row['tweet_text']).translate(non_bmp_map)
        tweet_text = row['tweet_text'][0:50]
        print ('{:<3}{:18s}{}'.format(row['_id'], row['tweeter'], tweet_text))
    print ('\nverify JSON:')
    read_data = read_json(f)
    for i, p in enumerate(read_data):
        if i < 3:
            p['tweet_text'] = str(p['tweet_text']).translate(non_bmp_map)
            tweet_text = p['tweet_text'][0:50]
            print ('{:<3}{:18s}{}'.format(p['_id'], p['tweeter'], tweet_text))

### Web Scraping

In [ ]:
from bs4 import BeautifulSoup
import requests, json

def build_title(t):
    t = t.text
    t = t.split()
    ls = []
    for row in t:
        if row != '-':
            ls.append(row)
        elif row == '-':
            break
    return ' '.join(ls)

def release_date(r):
    r = r.text
    r = r.split()
    prefix = r[0] + s + r[1]
    if len(r) == 5:
        date = r[2] + s + r[3] + s + r[4]
    else:
        date = r[2] + s + r[3]
    return prefix, date        

def write_json(f, d):
    with open(f, 'w') as fout:
        json.dump(d, fout)

def read_json(f):
    with open(f) as f:
        return json.load(f)

if __name__ == '__main__':
    s = ' '
    dic_ls = []
    base_url = "https://ssearch.oreilly.com/?q=data+science"
    soup = BeautifulSoup(requests.get(base_url).text, 'lxml')
    books = soup.find_all('article')
    for i, row in enumerate(books):
        dic = {}
        tag = row.name
        tag_val = row['class']
        title = row.find('p', {'class' : 'title'})
        title = build_title(title)
        url = row.find('a', {'class' : 'learn-more'})
        learn_more = url.get('href')
        author = row.find('p', {'class' : 'note'}).text
        release = row.find('p', {'class' : 'note date2'})
        prefix, date = release_date(release)
        if len(tag_val) == 2:
            publisher = row.find('p', {'class' : 'note publisher'}).text
            item = row.find('img', {'class' : 'book'})
            cat = item.get('class')[0]
        else:
            publisher, cat = None, None
            desc = row.find('p', {'class' : 'description'}).text.split()
            desc = [row for i, row in enumerate(desc) if i < 7]
            desc = ' '.join(desc) + ' ...'
        dic['title'] = title
        dic['learn_more'] = learn_more
        if author[0:3] != 'Pub':
            dic['author'] = author
        if publisher is not None:
            dic['publisher'] = publisher
            dic['category'] = cat
        else:
            dic['event'] = desc 
        dic['date'] = date
        dic_ls.append(dic)
    f = 'data/scraped.json'
    write_json(f, dic_ls)
    data = read_json(f)
    for i, row in enumerate(data):
        if i < 6:
            print (row['title'])
            if 'author' in row.keys():
                print (row['author'])
            if 'publisher' in row.keys():
                print (row['publisher'])
            if 'category' in row.keys():
                print ('Category:', row['category'])
                print ('Release Date:', row['date'])
            if 'event' in row.keys():
                print ('Event:', row['event'])
                print ('Publish Date:', row['date'])
            print ('Learn more:', row['learn_more'])
            print ()

In [ ]:
%reload_ext watermark
%watermark -a "Caique Miranda" -gu "caiquemiranda" -iv

### End.